# Environment Setup
Install and import all the necessary libraries: pandas, numpy, itertools, matplotlib, seaborn, sklearn, BeautifulSoup4, requests, lxml, and html5lib.

In [18]:
# NCAA Basketball Data Pipeline
# This notebook collects and processes the following data for March Madness teams:
# - season, point_differential, win%, efg, TS%, OReb, DReb, off efficiency, def efficiency, 
# - turnover ratio %, FTR, and Vegas odds
%pip install pandas
%pip install numpy
%pip install itertools
%pip install matplotlib
%pip install seaborn
%pip install sklearn
%pip install BeautifulSoup4
%pip install requests
%pip install lxml
%pip install html5lib
%pip install tqdm
%pip install ipywidgets


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.5_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.5_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement itertools (from versions: none)

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.5_1/libexec/bin/python -m pip install --upgrade pip
ERROR: No matching distribution found for itertools
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.5_1/libexec/bin

In [19]:
%jupyter nbextension enable --py widgetsnbextension

UsageError: Line magic function `%jupyter` not found.


In [10]:
# NCAA Basketball Data Pipeline
# This notebook collects and processes the following data for March Madness teams:
# - season, point_differential, win%, efg, TS%, OReb, DReb, off efficiency, def efficiency, 
# - turnover ratio %, FTR, and Vegas odds

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
import time
import random
from tqdm.notebook import tqdm
import re
import warnings
warnings.filterwarnings('ignore')

# Set paths
DATA_DIR = "../data"
OUTPUT_DIR = "../data/processed"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [11]:
# Load team data files
try:
    mens_teams = pd.read_csv(f"{DATA_DIR}/MTeams.csv")
    print(f"Loaded {len(mens_teams)} men's teams")
except FileNotFoundError:
    print("MTeams.csv not found. Creating dummy dataframe")
    mens_teams = pd.DataFrame({
        'TeamID': range(1101, 1121),
        'TeamName': [f"Team_{i}" for i in range(1101, 1121)]
    })

try:
    womens_teams = pd.read_csv(f"{DATA_DIR}/WTeams.csv")
    print(f"Loaded {len(womens_teams)} women's teams")
except FileNotFoundError:
    print("WTeams.csv not found. Creating dummy dataframe")
    womens_teams = pd.DataFrame({
        'TeamID': range(3101, 3121),
        'TeamName': [f"Team_{i}" for i in range(3101, 3121)]
    })

# Display sample rows
print("\nMen's Teams Sample:")
display(mens_teams.head())

print("\nWomen's Teams Sample:")
display(womens_teams.head())

Loaded 380 men's teams
Loaded 378 women's teams

Men's Teams Sample:


,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2025
1,1102,Air Force,1985,2025
2,1103,Akron,1985,2025
3,1104,Alabama,1985,2025
4,1105,Alabama A&M,2000,2025



Women's Teams Sample:


,TeamID,TeamName
0,3101,Abilene Chr
1,3102,Air Force
2,3103,Akron
3,3104,Alabama
4,3105,Alabama A&M


In [12]:
# Define additional files to load if available
supplementary_files = [
    "MRegularSeasonCompactResults.csv",
    "WRegularSeasonCompactResults.csv",
    "MTeamConferences.csv",
    "WTeamConferences.csv"
]

# Load supplementary data
loaded_data = {}
for file in supplementary_files:
    try:
        loaded_data[file] = pd.read_csv(f"{DATA_DIR}/{file}")
        print(f"Loaded {file}: {len(loaded_data[file])} rows")
    except FileNotFoundError:
        print(f"{file} not found. Skipping.")

# Try to identify available seasons
seasons = set()
for file_name, df in loaded_data.items():
    if 'Season' in df.columns:
        seasons.update(df['Season'].unique())

if seasons:
    print(f"\nAvailable seasons: {sorted(seasons)}")
else:
    # Default seasons if none found
    seasons = list(range(2015, 2025))
    print(f"\nNo seasons found in data. Using default range: {seasons}")

Loaded MRegularSeasonCompactResults.csv: 192497 rows
Loaded WRegularSeasonCompactResults.csv: 136628 rows
Loaded MTeamConferences.csv: 13388 rows
Loaded WTeamConferences.csv: 9490 rows

Available seasons: [np.int64(1985), np.int64(1986), np.int64(1987), np.int64(1988), np.int64(1989), np.int64(1990), np.int64(1991), np.int64(1992), np.int64(1993), np.int64(1994), np.int64(1995), np.int64(1996), np.int64(1997), np.int64(1998), np.int64(1999), np.int64(2000), np.int64(2001), np.int64(2002), np.int64(2003), np.int64(2004), np.int64(2005), np.int64(2006), np.int64(2007), np.int64(2008), np.int64(2009), np.int64(2010), np.int64(2011), np.int64(2012), np.int64(2013), np.int64(2014), np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024), np.int64(2025)]


In [13]:
# Functions for scraping and processing basketball-reference data
def fetch_sports_reference_team_data(team_name, season):
    """Fetch team statistics from basketball-reference.com"""
    # Format team name for URL (lowercase, replace spaces with hyphens)
    team_url_name = team_name.lower().replace(' ', '-').replace('&', '')
    url = f"https://www.sports-reference.com/cbb/schools/{team_url_name}/{season}.html"
    
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
    }
    
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            return parse_team_stats(soup, team_name, season)
        else:
            print(f"Failed to fetch {team_name} ({season}): Status code {response.status_code}")
            return None
    except Exception as e:
        print(f"Error fetching {team_name} ({season}): {e}")
        return None
    
def parse_team_stats(soup, team_name, season):
    """Parse basketball-reference HTML to extract team statistics"""
    data = {
        'team_name': team_name,
        'season': season,
        'wins': None,
        'losses': None,
        'win_pct': None,
        'points_for': None,
        'points_against': None,
        'point_differential': None,
        'efg_pct': None,
        'ts_pct': None,
        'oreb': None,
        'dreb': None,
        'off_efficiency': None,
        'def_efficiency': None,
        'turnover_pct': None,
        'ftr': None
    }
    
    # Extract record (wins-losses)
    try:
        record_text = soup.select_one('p:-soup-contains("Record:")').text
        match = re.search(r'Record: (\d+)-(\d+)', record_text)
        if match:
            data['wins'] = int(match.group(1))
            data['losses'] = int(match.group(2))
            data['win_pct'] = data['wins'] / (data['wins'] + data['losses'])
    except (AttributeError, ValueError) as e:
        print(f"Error extracting record for {team_name} ({season}): {e}")
    
    # Extract team stats from tables
    try:
        # Find all tables on the page
        tables = soup.find_all('table')
        
        # Per Game Stats
        for table in tables:
            if table.get('id') == 'team_stats':
                rows = table.find_all('tr')
                for row in rows:
                    if row.find('th') and 'team' in row.find('th').text.lower():
                        cells = row.find_all('td')
                        if len(cells) >= 22:  # Adjust based on actual table structure
                            data['points_for'] = float(cells[-2].text) if cells[-2].text else None
                            data['points_against'] = float(cells[-1].text) if cells[-1].text else None
                            if data['points_for'] and data['points_against']:
                                data['point_differential'] = data['points_for'] - data['points_against']
        
        # Advanced Stats
        for table in tables:
            if table.get('id') == 'team_advanced':
                rows = table.find_all('tr')
                for row in rows:
                    if row.find('th') and 'team' in row.find('th').text.lower():
                        cells = row.find_all('td')
                        if len(cells) >= 20:  # Adjust based on actual table structure
                            # These indices may need adjustment based on the table structure
                            data['efg_pct'] = float(cells[8].text) if cells[8].text and cells[8].text != '' else None
                            data['ts_pct'] = float(cells[9].text) if cells[9].text and cells[9].text != '' else None
                            data['off_efficiency'] = float(cells[1].text) if cells[1].text and cells[1].text != '' else None
                            data['def_efficiency'] = float(cells[2].text) if cells[2].text and cells[2].text != '' else None
                            data['turnover_pct'] = float(cells[12].text) if cells[12].text and cells[12].text != '' else None
                            data['ftr'] = float(cells[10].text) if cells[10].text and cells[10].text != '' else None
        
        # Totals for rebounds
        for table in tables:
            if table.get('id') == 'team_totals':
                rows = table.find_all('tr')
                for row in rows:
                    if row.find('th') and 'team' in row.find('th').text.lower():
                        cells = row.find_all('td')
                        if len(cells) >= 22:  # Adjust based on actual table structure
                            data['oreb'] = float(cells[12].text) if cells[12].text and cells[12].text != '' else None
                            data['dreb'] = float(cells[13].text) if cells[13].text and cells[13].text != '' else None
                            
    except Exception as e:
        print(f"Error parsing stats for {team_name} ({season}): {e}")
    
    return data

def fetch_vegas_odds(team_name, season):
    """Fetch Vegas odds data (placeholder - would need specific API or source)"""
    # This would be implemented with a specific data source
    # Returning placeholder for now
    return {
        'team_name': team_name,
        'season': season,
        'vegas_odds': random.uniform(5, 100)  # Placeholder random odds
    }

In [20]:
# Modified function to use standard tqdm instead of notebook tqdm
def process_teams_data(teams_df, gender='M', max_teams=None, start_season=None, end_season=None):
    """Process data for a set of teams across multiple seasons"""
    results = []
    
    # Determine seasons to process
    if start_season and end_season:
        seasons_to_process = list(range(start_season, end_season + 1))
    else:
        seasons_to_process = sorted(seasons)
    
    # Limit number of teams for testing if needed
    if max_teams:
        teams_subset = teams_df.head(max_teams)
    else:
        teams_subset = teams_df
        
    # Progress tracking
    total_combinations = len(teams_subset) * len(seasons_to_process)
    
    print(f"Processing {len(teams_subset)} {gender} teams across {len(seasons_to_process)} seasons...")
    
    # Use standard tqdm instead of tqdm.notebook
    from tqdm import tqdm
    
    # Process each team for each season
    counter = 0
    for _, team_row in teams_subset.iterrows():
        team_id = team_row['TeamID']
        team_name = team_row['TeamName']
        
        for season in seasons_to_process:
            # Fetch team statistics
            team_stats = fetch_sports_reference_team_data(team_name, season)
            
            # Fetch Vegas odds (placeholder)
            odds_data = fetch_vegas_odds(team_name, season)
            
            # Combine data
            if team_stats:
                combined_data = {
                    'TeamID': team_id,
                    'gender': gender,
                    **team_stats
                }
                
                # Add Vegas odds if available
                if odds_data and 'vegas_odds' in odds_data:
                    combined_data['vegas_odds'] = odds_data['vegas_odds']
                
                results.append(combined_data)
            
            # Add random delay to avoid rate limiting
            time.sleep(random.uniform(1, 3))
            counter += 1
            
            # Print progress manually
            print(f"Processed {counter}/{total_combinations} ({counter/total_combinations*100:.1f}%)", end="\r")
    
    print("\nProcessing complete!")
    return pd.DataFrame(results)

In [21]:
# Process a small subset for testing
# Adjust max_teams and seasons as needed
mens_data = process_teams_data(mens_teams, gender='M', max_teams=5, start_season=2023, end_season=2024)
womens_data = process_teams_data(womens_teams, gender='W', max_teams=5, start_season=2023, end_season=2024)

print("\nMen's data sample:")
display(mens_data.head())

print("\nWomen's data sample:")
display(womens_data.head())

Processing 5 M teams across 2 seasons...
Failed to fetch Abilene Chr (2023): Status code 404
Failed to fetch Abilene Chr (2024): Status code 404
Processed 2/10 (20.0%)

Exception ignored in: <function tqdm.__del__ at 0x11c2f6020>
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/jupyterlab/4.3.5_1/libexec/lib/python3.13/site-packages/tqdm/std.py", line 1148, in __del__
    self.close()
  File "/opt/homebrew/Cellar/jupyterlab/4.3.5_1/libexec/lib/python3.13/site-packages/tqdm/notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
AttributeError: 'tqdm_notebook' object has no attribute 'disp'


Processed 10/10 (100.0%)
Processing complete!
Processing 5 W teams across 2 seasons...
Failed to fetch Abilene Chr (2023): Status code 404
Failed to fetch Abilene Chr (2024): Status code 404
Failed to fetch Alabama (2023): Status code 429
Failed to fetch Alabama (2024): Status code 429
Failed to fetch Alabama A&M (2023): Status code 429
Failed to fetch Alabama A&M (2024): Status code 429
Processed 10/10 (100.0%)
Processing complete!

Men's data sample:


,TeamID,gender,team_name,season,wins,losses,win_pct,points_for,points_against,point_differential,efg_pct,ts_pct,oreb,dreb,off_efficiency,def_efficiency,turnover_pct,ftr,vegas_odds
0,1102,M,Air Force,2023,14,18,0.437500,None,None,None,None,None,None,None,None,None,None,None,74.794004
1,1102,M,Air Force,2024,9,22,0.290323,None,None,None,None,None,None,None,None,None,None,None,71.036142
2,1103,M,Akron,2023,22,11,0.666667,None,None,None,None,None,None,None,None,None,None,None,35.986168
3,1103,M,Akron,2024,24,11,0.685714,None,None,None,None,None,None,None,None,None,None,None,72.524650
4,1104,M,Alabama,2023,31,6,0.837838,None,None,None,None,None,None,None,None,None,None,None,17.832050



Women's data sample:


,TeamID,gender,team_name,season,wins,losses,win_pct,points_for,points_against,point_differential,efg_pct,ts_pct,oreb,dreb,off_efficiency,def_efficiency,turnover_pct,ftr,vegas_odds
0,3102,W,Air Force,2023,14,18,0.437500,None,None,None,None,None,None,None,None,None,None,None,36.742259
1,3102,W,Air Force,2024,9,22,0.290323,None,None,None,None,None,None,None,None,None,None,None,74.540160
2,3103,W,Akron,2023,22,11,0.666667,None,None,None,None,None,None,None,None,None,None,None,71.966970
3,3103,W,Akron,2024,24,11,0.685714,None,None,None,None,None,None,None,None,None,None,None,14.299561


In [ ]:

# Check for missing data
def check_data_quality(df, gender):
    """Check for missing values and data quality issues"""
    print(f"\n{gender}'s Data Quality Check:")
    
    # Missing values per column
    missing = df.isnull().sum()
    print("Missing values per column:")
    print(missing[missing > 0])
    
    # Completeness percentage
    completeness = (1 - df.isnull().mean()) * 100
    print("\nCompleteness percentage per column:")
    print(completeness)
    
    # Basic statistics for numerical columns
    print("\nBasic statistics:")
    print(df.describe().T)

check_data_quality(mens_data, "Men's")
check_data_quality(womens_data, "Women's")

In [ ]:
# Function to process all teams with recovery
def process_all_teams(mens_teams, womens_teams, start_season=None, end_season=None, 
                     checkpoint_interval=10, recovery_file='checkpoint.pkl'):
    """Process all teams with checkpointing for recovery"""
    
    # Check for recovery file
    if os.path.exists(recovery_file):
        print(f"Recovery file found. Loading data from {recovery_file}")
        try:
            checkpoint = pd.read_pickle(recovery_file)
            mens_data = checkpoint.get('mens_data', pd.DataFrame())
            womens_data = checkpoint.get('womens_data', pd.DataFrame())
            processed_teams = checkpoint.get('processed_teams', set())
            print(f"Loaded {len(mens_data)} men's records and {len(womens_data)} women's records")
        except Exception as e:
            print(f"Error loading recovery file: {e}")
            mens_data = pd.DataFrame()
            womens_data = pd.DataFrame()
            processed_teams = set()
    else:
        mens_data = pd.DataFrame()
        womens_data = pd.DataFrame()
        processed_teams = set()
    
    # Process men's teams
    remaining_mens_teams = mens_teams[~mens_teams['TeamID'].isin(processed_teams)]
    print(f"Processing {len(remaining_mens_teams)} remaining men's teams")
    
    if len(remaining_mens_teams) > 0:
        new_mens_data = process_teams_data(remaining_mens_teams, gender='M', 
                                          start_season=start_season, end_season=end_season)
        mens_data = pd.concat([mens_data, new_mens_data], ignore_index=True)
        processed_teams.update(remaining_mens_teams['TeamID'])
        
        # Save checkpoint
        checkpoint = {
            'mens_data': mens_data,
            'womens_data': womens_data,
            'processed_teams': processed_teams
        }
        pd.to_pickle(checkpoint, recovery_file)
        print(f"Men's data checkpoint saved, {len(mens_data)} records")
    
    # Process women's teams
    remaining_womens_teams = womens_teams[~womens_teams['TeamID'].isin(processed_teams)]
    print(f"Processing {len(remaining_womens_teams)} remaining women's teams")
    
    if len(remaining_womens_teams) > 0:
        new_womens_data = process_teams_data(remaining_womens_teams, gender='W', 
                                            start_season=start_season, end_season=end_season)
        womens_data = pd.concat([womens_data, new_womens_data], ignore_index=True)
        processed_teams.update(remaining_womens_teams['TeamID'])
        
        # Save checkpoint
        checkpoint = {
            'mens_data': mens_data,
            'womens_data': womens_data,
            'processed_teams': processed_teams
        }
        pd.to_pickle(checkpoint, recovery_file)
        print(f"Women's data checkpoint saved, {len(womens_data)} records")
    
    return mens_data, womens_data

In [ ]:
# Uncomment to process all teams (this will take a long time)
# Define seasons range
start_season = 2015
end_season = 2024

# Process all teams
# full_mens_data, full_womens_data = process_all_teams(
#     mens_teams, womens_teams, 
#     start_season=start_season, 
#     end_season=end_season
# )

In [ ]:
# Save the processed data to CSV
def save_processed_data(mens_data, womens_data):
    """Save the processed data to CSV files"""
    # Save men's data
    mens_output_path = f"{OUTPUT_DIR}/mens_basketball_metrics.csv"
    mens_data.to_csv(mens_output_path, index=False)
    print(f"Men's data saved to {mens_output_path}")
    
    # Save women's data
    womens_output_path = f"{OUTPUT_DIR}/womens_basketball_metrics.csv"
    womens_data.to_csv(womens_output_path, index=False)
    print(f"Women's data saved to {womens_output_path}")
    
# Save test data for now
save_processed_data(mens_data, womens_data)

# To save the full data when ready, uncomment:
# save_processed_data(full_mens_data, full_womens_data)

In [ ]:
# Basic data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Combine men's and women's data for comparison
mens_data['league'] = 'Men'
womens_data['league'] = 'Women'
combined_data = pd.concat([mens_data, womens_data], ignore_index=True)

# Set up the visualization style
plt.style.use('seaborn-v0_8-whitegrid')
plt.figure(figsize=(15, 10))

# Example visualizations
metrics = ['win_pct', 'point_differential', 'efg_pct', 'ts_pct', 'off_efficiency', 'def_efficiency']

for i, metric in enumerate(metrics, 1):
    plt.subplot(2, 3, i)
    
    # Skip if the metric has too many missing values
    if combined_data[metric].isna().mean() > 0.5:
        plt.text(0.5, 0.5, f"Insufficient data\nfor {metric}", 
                 horizontalalignment='center', verticalalignment='center', fontsize=12)
        plt.axis('off')
        continue
    
    # Create boxplots grouped by league
    sns.boxplot(x='league', y=metric, data=combined_data)
    plt.title(f'{metric} by League')
    plt.xlabel('')
    plt.tight_layout()

plt.suptitle('Basketball Metrics Comparison: Men vs Women', fontsize=16, y=1.05)
plt.tight_layout()
plt.show()